# Find Peaks Library Performance and Validation

This notebook compares the C and C++ implementations of the Find Peaks library against SciPy's reference implementation. It validates correctness and benchmarks performance.

## Setup and Imports

In [ ]:
import sys
import numpy as np
from scipy.signal import find_peaks

# Add the path to our Python bindings
# # For example:
# sys.path.insert(0, '../cmake-build-release/bindings/')

sys.path.insert(0, '<path_to_your_cpp_bindings>') # Adjust this path to your setup (where the .pyd / .so is)

# Import our C and C++ implementations
from find_peaks_wrapper import find_peaks_cpp, find_peaks_c

## Test Data Generation

We'll create a large random signal for testing and define functions to generate random parameter sets.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Generate test signal with 100,000 points
x = np.random.randn(100000)

In [ ]:
def generate_params():
    """Generate random parameter sets for peak detection algorithms.
    
    Returns:
        dict: Dictionary of random parameter values for find_peaks functions
    """
    d = {}
    d['height'] = np.random.rand()*5
    d['threshold'] = np.random.rand()*5
    d['distance'] = 1 + int(np.random.randint(0, 100))
    d['prominence'] = np.random.rand()*5
    d['width'] = np.random.rand()*5
    d['wlen'] = 2 + int(np.random.rand()*5)
    d['rel_height'] = np.random.rand()
    d['plateau_size'] = int(np.random.randint(0, 100))

    return d

In [ ]:
def compare_results(r1, r2):
    """Compare results from different peak detection implementations.
    
    Args:
        r1: Result tuple from first implementation (peaks, properties)
        r2: Result tuple from second implementation (peaks, properties)
        
    Returns:
        bool: True if results match, False otherwise
    """
    # Find common property keys between the two results
    common_keys = set(r1[1].keys()).intersection(set(r2[1].keys()))
    
    # Check if peak indices match
    if r1[0].shape != r2[0].shape:
        return False
    
    # Compare peak indices and all common properties
    comp1 = np.all(r1[0] == r2[0])
    comp2 = np.all([np.all(r1[1][key] == r2[1][key]) for key in common_keys])
    
    return comp1 and comp2

## Validation Testing

Run multiple tests with random parameters to validate that our C and C++ implementations produce the same results as SciPy's reference implementation.

In [ ]:
print("Running validation tests with 100 different parameter sets...")
mismatches = []

for i in range(100):
    p = generate_params()
    
    scipy_result = find_peaks(x, **p)
    c_result = find_peaks_c(x, **p)
    cpp_result = find_peaks_cpp(x, **p)
    
    if not compare_results(scipy_result, c_result) or not compare_results(scipy_result, cpp_result):
        mismatches.append(p)

if mismatches:
    print(f"Found {len(mismatches)} parameter sets that produced mismatches:")
    for i, params in enumerate(mismatches):
        print(f"Mismatch {i+1}: {params}")
else:
    print("All tests passed! All implementations produce the same results.")

## Performance Benchmarking

Compare the performance of SciPy's implementation against our C and C++ implementations.

In [ ]:
# Define a consistent set of parameters for benchmarking
benchmark_params = {
    'height': 1.6, 
    'threshold': 0.5, 
    'distance': 53, 
    'prominence': 3.0, 
    'width': 2.8, 
    'wlen': 30, 
    'rel_height': 0.86, 
    'plateau_size': 1
}

In [ ]:
%%time
print("Benchmarking SciPy implementation...")
for i in range(10):
    scipy_peaks = find_peaks(x, **benchmark_params)
print(f"Found {len(scipy_peaks[0])} peaks")

In [ ]:
%%time
print("Benchmarking C implementation...")
for i in range(10):
    c_peaks = find_peaks_c(x, **benchmark_params)
print(f"Found {len(c_peaks[0])} peaks")

In [ ]:
%%time
print("Benchmarking C++ implementation...")
for i in range(10):
    cpp_peaks = find_peaks_cpp(x, **benchmark_params)
print(f"Found {len(cpp_peaks[0])} peaks")

## Result Verification

Verify that all implementations find the same number of peaks and the same peak locations.

In [ ]:
# Get results from all implementations
scipy_result = find_peaks(x, **benchmark_params)
c_result = find_peaks_c(x, **benchmark_params)
cpp_result = find_peaks_cpp(x, **benchmark_params)

# Print peak counts
print(f"SciPy implementation found {len(scipy_result[0])} peaks")
print(f"C implementation found {len(c_result[0])} peaks")
print(f"C++ implementation found {len(cpp_result[0])} peaks")

In [ ]:
# Verify results match
print(f"SciPy and C++ results match: {compare_results(scipy_result, cpp_result)}")
print(f"SciPy and C results match: {compare_results(scipy_result, c_result)}")
print(f"C and C++ results match: {compare_results(c_result, cpp_result)}")

## Examine Result Properties

Take a closer look at the output properties from each implementation to ensure they match.

In [ ]:
# Print available property keys in each result
print("SciPy properties:", list(scipy_result[1].keys()))
print("C++ properties:", list(cpp_result[1].keys()))
print("C properties:", list(c_result[1].keys()))

In [ ]:
# Get the common property keys
common_keys = set(scipy_result[1].keys()).intersection(
                set(c_result[1].keys())).intersection(
                set(cpp_result[1].keys()))

print(f"Common property keys found in all implementations: {common_keys}")

In [ ]:
# Compare property values for a subset of the first few peaks
n_peaks_to_show = min(5, len(scipy_result[0]))

print(f"Comparing properties for the first {n_peaks_to_show} peaks:")
print("\nPeak indices:")
print(f"SciPy: {scipy_result[0][:n_peaks_to_show]}")
print(f"C++: {cpp_result[0][:n_peaks_to_show]}")
print(f"C: {c_result[0][:n_peaks_to_show]}")

print("\nProminences:")
print(f"SciPy: {scipy_result[1]['prominences'][:n_peaks_to_show]}")
print(f"C++: {cpp_result[1]['prominences'][:n_peaks_to_show]}")
print(f"C: {c_result[1]['prominences'][:n_peaks_to_show]}")

## Conclusion

This notebook validates that our C and C++ implementations of the find_peaks algorithm produce results identical to SciPy's reference implementation across a wide range of parameters.

Performance benchmarking results show how the native implementations compare to the Python version. The C++ implementation typically provides the best performance, followed by the C implementation, both outperforming the Python/SciPy version.